In [1]:
import pandas as pd
import glob
from os.path import basename
from os.path import exists
from datetime import date

In [2]:
CODIGO_MATERIA = 'O'
ASIGNATURA = 'DESARROLLO DE SOFTWARE'
NUMERO_GRUPOS = 59
RUTA = './INPUT/GRUPOS_PREDEFINITIVAS/'
CONSULTA = './INPUT/DEFINITIVAS-PROG-23-09.csv'
MATERIA_CON_HOMOLOGACION = False
GRUPO_HOMOLOGACION = 'HOM'
NOMENCLATURA_OMITIR = 5
NOMENCLATURA_TIENE_PUNTOS = False
TOLERANCIA_MOODLE = 0.051
ACTIVIDADES_CALIFICABLES = [
    'Sprint 1. Definiendo el Proyecto',
    'Sprint 2. Diseñando la Base de Datos',
    'Sprint 3. Estructurando el Backend',
    'Sprint 4. Estructurando el FrontEnd'
]

In [3]:
DEFINITIVAS = pd.read_csv(CONSULTA, skiprows=1, sep=';', encoding='latin-1')
DEFINITIVAS = DEFINITIVAS[DEFINITIVAS['ASIGNATURA'] == ASIGNATURA]

In [4]:
def extraerNombre(ruta, punto = False):
    nBase = basename(archivo) 
    tokens = nBase.split('-')
    casiNombre = '-'.join(tokens[NOMENCLATURA_OMITIR:-1])
    nombre = casiNombre.split('.')[0]
    return nombre if not(punto) else casiNombre

def getNotaLibro(notaLibro, rayita = True):
    if notaLibro == '-':
        return notaLibro if rayita else 0.0
    return float(notaLibro)

def buscarEnDf(df, columnaKey, key, columna):
    query = df[df[columnaKey] == key][columna]
    if len(query) > 0:        
        return query.iloc[0]
    return -1

In [5]:
if MATERIA_CON_HOMOLOGACION:
    DEFINITIVAS_HOMOLOGADOS = DEFINITIVAS[DEFINITIVAS['GRUPO'] == GRUPO_HOMOLOGACION]
noEncontrados = []
for i in range (1, NUMERO_GRUPOS + 1):
    num = str(i)
    grupo = CODIGO_MATERIA + num
    rutaCarpeta = RUTA + grupo
    if not exists(rutaCarpeta):
        print (f'El curso {grupo} no tiene una carpeta definida, omitido')
        continue
    archivosCarpeta = glob.glob(rutaCarpeta + '/*.csv')
    df = -1
    for archivo in archivosCarpeta:
        nombreArchivo = extraerNombre(archivo, NOMENCLATURA_TIENE_PUNTOS)
        if not(nombreArchivo in ACTIVIDADES_CALIFICABLES):
            df = pd.read_csv(archivo)
            break
    if type(df) == int:
        raise Exception('Archivo de calificaciones no encontrado')
    definitivasGrupo = DEFINITIVAS[DEFINITIVAS['GRUPO'] == grupo]
    for index, row in df.iterrows():
        nota = round(getNotaLibro(row['Total del curso (Real)'], False), 1)
        codBase = row['Número de ID'] if type(row['Número de ID']) == str else '-1'
        codigoEst = int(codBase.split('-')[-1])
        notaSA = buscarEnDf(definitivasGrupo, 'ESTUDIANTE', codigoEst, 'NOTA')
        if nota != notaSA:
            homologado = False
            if MATERIA_CON_HOMOLOGACION:
                notaHom = buscarEnDf(DEFINITIVAS_HOMOLOGADOS, 'ESTUDIANTE', codigoEst, 'NOTA')
                if notaHom != -1:
                    homologado = True
            noEncontrados.append([codigoEst, grupo, homologado, nota, notaSA])
        

In [6]:
nNoEncontrados = len(noEncontrados)
print (nNoEncontrados)
if nNoEncontrados > 0:
    pd.DataFrame(noEncontrados, columns = ['CÓDIGO', 'GRUPO', 'HOMOLOGADO', 'NOTA_LIBRO', 'NOTA_SISTEMA_ACADÉMICO']).to_excel('./OUTPUT/AUDITORIA_NOTAS_SA_'+ ASIGNATURA + '_' + str(date.today()) + '.xlsx', index = False)

68
